In [1]:
import jax.numpy as np
from jax import random, grad, vmap, jit
from jax.config import config

import numpy as onp

import matplotlib

import matplotlib.pyplot as plt

import scipy.io
from scipy.interpolate import griddata


In [2]:
plt.rcParams.update(plt.rcParamsDefault)
plt.rc('font', family='serif')
plt.rcParams.update({
                      "text.usetex": True,
                      "font.family": "serif",
                     'text.latex.preamble': r'\usepackage{amsmath}',
                      'font.size': 16,
                      'lines.linewidth': 3,
                      'axes.labelsize': 20, 
                      'axes.titlesize': 16,
                      'xtick.labelsize': 16,
                      'ytick.labelsize': 16,
                      'legend.fontsize': 16,
                      'axes.linewidth': 2})



# Load the training data

usol = np.load("usol.npy")
usol = np.array(usol)

u_initial = np.load("u.npy")
u_initial = np.array(u_initial)

N = usol.shape[0]  # number of total input samples
N_train =1000      # number of input samples used for training
N_test = N - N_train  # number of input samples used for test

# Resolution of the solution
Nx = 100
Nt = 100
m = Nx            # number of sensors for input samples
P = 100       # resolution of uniform grid for the data

u0_train = u_initial[:N_train,:]   # input samples
usol_train = usol[:N_train,:,:]

u0_test = u_initial[N_train:N,:]   # input samples
usol_test = usol[N_train:N,:,:]

print("shape of training data", u0_train.shape, usol_train.shape)
print("shape of testing data", u0_test.shape, usol_test.shape)


shape of training data (1000, 100) (1000, 100, 100)
shape of testing data (1000, 100) (1000, 100, 100)


In [3]:
Predict_mu_save = np.load("Predict_mu_save.npy")
Predict_std_save = np.load("Predict_std_save.npy")
errors = np.load("normed_errors.npy")

print(Predict_mu_save.shape, Predict_std_save.shape, errors.shape)

(1000, 100, 100) (1000, 100, 100) (1000, 1)


In [4]:
uncertainty = onp.zeros_like(errors)

In [5]:
idxs = [30, 130, 230, 330, 430, 530, 630, 730, 830, 930]
t = np.linspace(0, 1, P)
x = np.linspace(0, 1, P)
T, X = np.meshgrid(t, x)


for k in range(N_test):

    idx = k + N_train
    u = usol[idx,:, :]

    y_test_sample = np.hstack([T.flatten()[:,None], X.flatten()[:,None]])
    s_test_sample = u.flatten()[:,None]


    S_pred_sample_mu = Predict_mu_save[k,:,:]
    S_pred_sample_std = Predict_std_save[k,:,:]

    uncertainty[k,0] = np.linalg.norm(S_pred_sample_std, 2) / np.linalg.norm(u, 2)
    
    if k in idxs:

        fig = plt.figure(figsize=(12,9))
        plt.subplot(2,2,1)
        plt.pcolor(T, X, u, cmap='jet')
        plt.xlabel('$x$')
        plt.ylabel('$t$')
        plt.title('Exact $s(x,t)$')
        plt.colorbar()
        plt.tight_layout()

        plt.subplot(2,2,2)
        plt.pcolor(T, X, S_pred_sample_mu, cmap='jet')
        plt.xlabel('$x$')
        plt.ylabel('$t$')
        plt.title('Predictive mean of $s(x,t)$')
        plt.colorbar()
        plt.tight_layout()

        plt.subplot(2,2,3)
        plt.pcolor(T, X, np.abs(S_pred_sample_mu - u), cmap='jet')
        plt.xlabel('$x$')
        plt.ylabel('$t$')
        plt.title('Absolute error')
        plt.colorbar()
        plt.tight_layout()

        plt.subplot(2,2,4)
        plt.pcolor(T, X, S_pred_sample_std, cmap='jet')
        plt.xlabel('$x$')
        plt.ylabel('$t$')
        plt.title('Predictive uncertainty of $s(x,t)$')
        plt.colorbar()
        plt.tight_layout()
        plt.savefig('./normed_testing_Samples' + str(idx) + '.png', dpi = 300)
        
        
        fig = plt.figure(figsize=(15,4))
        plt.subplot(1,3,1)
        plt.plot(x,u[25,:], 'b-', linewidth = 2, label = 'Exact')       
        plt.plot(x,S_pred_sample_mu[25,:], 'r--', linewidth = 2, label = 'Prediction')
        lower = S_pred_sample_mu[25,:] - 2.0*S_pred_sample_std[25,:]
        upper = S_pred_sample_mu[25,:] + 2.0*S_pred_sample_std[25,:]
        plt.fill_between(x.flatten(), lower.flatten(), upper.flatten(), 
                        facecolor='orange', alpha=0.5, label="Two std band")
        plt.xlabel('$x$')
        plt.ylabel('$s(x)$')  
        plt.title('$t = 0.25$')
        # ax.set_xlim([-0.1,1.1])
        # ax.set_ylim([-1.1,1.1])

        plt.subplot(1,3,2)
        plt.plot(x,u[50,:], 'b-', linewidth = 2, label = 'Exact')       
        plt.plot(x,S_pred_sample_mu[50,:], 'r--', linewidth = 2, label = 'Prediction')
        lower = S_pred_sample_mu[50,:] - 2.0*S_pred_sample_std[50,:]
        upper = S_pred_sample_mu[50,:] + 2.0*S_pred_sample_std[50,:]
        plt.fill_between(x.flatten(), lower.flatten(), upper.flatten(), 
                        facecolor='orange', alpha=0.5, label="Two std band")
        plt.xlabel('$x$')
        plt.ylabel('$s(x)$')
        # ax.set_xlim([-0.1,1.1])
        # ax.set_ylim([-1.1,1.1])
        plt.title('$t = 0.50$')
        plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.35), ncol=5, frameon=False)

        plt.subplot(1,3,3)
        plt.plot(x,u[75,:], 'b-', linewidth = 2, label = 'Exact')       
        plt.plot(x,S_pred_sample_mu[75,:], 'r--', linewidth = 2, label = 'Prediction')
        lower = S_pred_sample_mu[75,:] - 2.0*S_pred_sample_std[75,:]
        upper = S_pred_sample_mu[75,:] + 2.0*S_pred_sample_std[75,:]
        plt.fill_between(x.flatten(), lower.flatten(), upper.flatten(), 
                        facecolor='orange', alpha=0.5, label="Two std band")
        plt.xlabel('$x$')
        plt.ylabel('$s(x)$')
        # ax.set_xlim([-0.1,1.1])
        # ax.set_ylim([-1.1,1.1])    
        plt.title('$t = 0.75$')
        plt.tight_layout(w_pad=-4.5)
        plt.savefig('./normed_testing_slices' + str(idx) + '.png')


In [6]:
idx_max = np.argmax(errors)
print("outlier index and error", idx_max, errors[idx_max,0])
idx = idx_max + N_train

u = usol[idx,:, :]
u0 = usol[idx,0,:]

u_test_sample = np.tile(u0, (P**2, 1))
y_test_sample = np.hstack([T.flatten()[:,None], X.flatten()[:,None]])
s_test_sample = u.flatten()[:,None]

S_pred_sample_mu = Predict_mu_save[idx_max,:,:]
S_pred_sample_std = Predict_std_save[idx_max,:,:]


fig = plt.figure(figsize=(12,9))
plt.subplot(2,2,1)
plt.pcolor(T, X, u, cmap='jet')
plt.xlabel('$x$')
plt.ylabel('$t$')
plt.title('Exact $s(x,t)$')
plt.colorbar()
plt.tight_layout()

plt.subplot(2,2,2)
plt.pcolor(T, X, S_pred_sample_mu, cmap='jet')
plt.xlabel('$x$')
plt.ylabel('$t$')
plt.title('Predictive mean of $s(x,t)$')
plt.colorbar()
plt.tight_layout()

plt.subplot(2,2,3)
plt.pcolor(T, X, np.abs(S_pred_sample_mu - u), cmap='jet')
plt.xlabel('$x$')
plt.ylabel('$t$')
plt.title('Absolute error')
plt.colorbar()
plt.tight_layout()

plt.subplot(2,2,4)
plt.pcolor(T, X, S_pred_sample_std, cmap='jet')
plt.xlabel('$x$')
plt.ylabel('$t$')
plt.title('Predictive uncertainty of $s(x,t)$')
plt.colorbar()
plt.tight_layout()
plt.savefig('./normed_testing_Samples_max.png', dpi = 300)


fig = plt.figure(figsize=(15,4))
plt.subplot(1,3,1)
plt.plot(x,u[25,:], 'b-', linewidth = 2, label = 'Exact')       
plt.plot(x,S_pred_sample_mu[25,:], 'r--', linewidth = 2, label = 'Prediction')
lower = S_pred_sample_mu[25,:] - 2.0*S_pred_sample_std[25,:]
upper = S_pred_sample_mu[25,:] + 2.0*S_pred_sample_std[25,:]
plt.fill_between(x.flatten(), lower.flatten(), upper.flatten(), 
                facecolor='orange', alpha=0.5, label="Two std band")
plt.xlabel('$x$')
plt.ylabel('$s(x)$')  
plt.title('$t = 0.25$')
# ax.set_xlim([-0.1,1.1])
# ax.set_ylim([-1.1,1.1])

plt.subplot(1,3,2)
plt.plot(x,u[50,:], 'b-', linewidth = 2, label = 'Exact')       
plt.plot(x,S_pred_sample_mu[50,:], 'r--', linewidth = 2, label = 'Prediction')
lower = S_pred_sample_mu[50,:] - 2.0*S_pred_sample_std[50,:]
upper = S_pred_sample_mu[50,:] + 2.0*S_pred_sample_std[50,:]
plt.fill_between(x.flatten(), lower.flatten(), upper.flatten(), 
                facecolor='orange', alpha=0.5, label="Two std band")
plt.xlabel('$x$')
plt.ylabel('$s(x)$')
# ax.set_xlim([-0.1,1.1])
# ax.set_ylim([-1.1,1.1])
plt.title('$t = 0.50$')
plt.legend(loc='upper center', bbox_to_anchor=(0.7, -0.35), ncol=5, frameon=False)

plt.subplot(1,3,3)
plt.plot(x,u[75,:], 'b-', linewidth = 2, label = 'Exact')       
plt.plot(x,S_pred_sample_mu[75,:], 'r--', linewidth = 2, label = 'Prediction')
lower = S_pred_sample_mu[75,:] - 2.0*S_pred_sample_std[75,:]
upper = S_pred_sample_mu[75,:] + 2.0*S_pred_sample_std[75,:]
plt.fill_between(x.flatten(), lower.flatten(), upper.flatten(), 
                facecolor='orange', alpha=0.5, label="Two std band")
plt.xlabel('$x$')
plt.ylabel('$s(x)$')
# ax.set_xlim([-0.1,1.1])
# ax.set_ylim([-1.1,1.1])    
plt.title('$t = 0.75$')
plt.tight_layout(w_pad=-7.5)
plt.savefig('./normed_testing_slices_max.png')


outlier index and error 360 0.05514690652489662


/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  app.launch_new_instance()
/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:51: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


In [7]:


plt.figure(figsize=(6,5))
plt.plot(errors[0], uncertainty[0], 'ro', alpha = 0.5, label = "In distribution samples")
plt.plot(errors, uncertainty, 'ro', alpha = 0.5)
plt.plot(errors[idx_max], uncertainty[idx_max], 'bo', alpha = 1, label = "Out of distribution sample") 
plt.legend(loc='upper left', frameon=False, prop={'size': 16})
plt.xlabel('Error')
plt.ylabel('Uncertainty')
plt.tight_layout()
plt.savefig('./Reaction_Error_vs_Uncertainty.png', dpi = 300)




/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


In [11]:
from scipy import stats

######## Probability density kernel estimation ########
xmin, xmax = errors.min(), errors.max()
X_marginal_1 = np.linspace(xmin, xmax, 100)[:,None]
positions_marginal_1 = X_marginal_1.flatten()
values_marginal_1 = errors.flatten()
gkde = stats.gaussian_kde(values_marginal_1)
KDE_marginal_1 = gkde.evaluate(positions_marginal_1)


xmin, xmax = uncertainty.min(), uncertainty.max()
X_marginal_3 = np.linspace(xmin, xmax, 100)[:,None]
positions_marginal_3 = X_marginal_3.flatten()
values_marginal_3 = uncertainty.flatten()
gkde = stats.gaussian_kde(values_marginal_3)
KDE_marginal_3 = gkde.evaluate(positions_marginal_3)


    

plt.figure(figsize=(6,5))
plt.hist(errors, bins = 100, density=True, alpha = 0.6, color='blue')
plt.hist(uncertainty, bins = 100, density=True, alpha = 0.6, color='red')
plt.plot(X_marginal_1, KDE_marginal_1, 'b-', label = 'Error')
plt.plot(X_marginal_3, KDE_marginal_3, 'r-', label = 'Uncertainty')
plt.xlabel('Error/Uncertainty')
plt.ylabel('Probability density')
plt.xlim(0, 0.06)
plt.legend(loc='upper right', frameon=False, prop={'size': 16})
plt.tight_layout()
plt.savefig('./Reaction_Error_vs_Uncertainty_marginal.png', dpi = 300)



/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


In [9]:
errors_star = errors / np.max(errors)
uncertainty_star = uncertainty / np.max(uncertainty)

######## Probability density kernel estimation ########
xmin, xmax = errors_star.min(), errors_star.max()
X_marginal_1 = np.linspace(xmin, xmax, 100)[:,None]
positions_marginal_1 = X_marginal_1.flatten()
values_marginal_1 = errors_star.flatten()
gkde = stats.gaussian_kde(values_marginal_1)
KDE_marginal_1 = gkde.evaluate(positions_marginal_1)


xmin, xmax = uncertainty_star.min(), uncertainty_star.max()
X_marginal_3 = np.linspace(xmin, xmax, 100)[:,None]
positions_marginal_3 = X_marginal_3.flatten()
values_marginal_3 = uncertainty_star.flatten()
gkde = stats.gaussian_kde(values_marginal_3)
KDE_marginal_3 = gkde.evaluate(positions_marginal_3)


print(errors_star.shape, uncertainty_star.shape)

plt.figure(figsize=(6,5))
plt.hist(errors_star.flatten(), bins = 100, density=True, alpha = 0.6, color='blue')
plt.hist(uncertainty_star.flatten(), bins = 100, density=True, alpha = 0.6, color='red')
plt.plot(X_marginal_1, KDE_marginal_1, 'b-', label = 'Scaled error')
plt.plot(X_marginal_3, KDE_marginal_3, 'r-', label = 'Scaled uncertainty')
plt.xlabel('Error/Uncertainty',fontsize=13)
plt.ylabel('Probability density',fontsize=13)
plt.legend(loc='upper right', frameon=False, prop={'size': 16})
plt.tight_layout()
plt.savefig('./Reaction_Error_vs_Uncertainty_marginal_star.png', dpi = 300)



(1000, 1) (1000, 1)


/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
